# Comparacion SVD implementado en CUDA

Carga de librerias e inicio de pycuda

In [1]:
import pycuda.autoinit
import pandas as pd
from sklearn.model_selection import train_test_split
import pycuda.gpuarray as gpuarray
import numpy as np
import skcuda.linalg as culinalg
import time

/home/luis/anaconda3/envs/my_env/lib/python3.7/site-packages/skcuda/cublas.py:283: UserWarning: creating CUBLAS context to get version number
  warnings.warn('creating CUBLAS context to get version number')


In [2]:
culinalg.init()

In [3]:
np.random.seed(0)

In [4]:
datatype = np.float64

Importar archivo con calificaciones (ratings)

In [5]:
###############test datos
# Conjunto de datos con un millón de calificaciones (cifra redondeada)
ratings = pd.read_csv(r'/home/luis/Sist_Rec/ratings_1m.csv')

# Separar conjunto 75% entrenamiento y 25% validación, se usa train_test_split de sklearn
train, test = train_test_split(ratings,test_size=0.25)


In [6]:
# El mapeo lo que hace es tener un id consecutivo y único de usuarios y de películas
# La tabla original de películas no tiene id consecutivos

lista_pelic = ratings['movieId'].unique()
num_pelic = len(lista_pelic)
mapeo_pelic = {}
for n, m in enumerate(lista_pelic):
    mapeo_pelic[m] = n
lista_usuarios = ratings['userId'].unique()
num_usuarios = len(lista_usuarios)
mapeo_usuarios = {}
for n, m in enumerate(lista_usuarios):
    mapeo_usuarios[m] = n

In [7]:
matriz_uspl = np.zeros([num_usuarios, num_pelic])
for index, row in ratings.iterrows():
    matriz_uspl[mapeo_usuarios[row['userId']], mapeo_pelic[row['movieId']]] = int(row['rating'])

# Se guarda una copia como Data Frame
matriz_df = np.asarray(matriz_uspl,datatype) 

matriz_df = np.transpose(matriz_df)

print(matriz_df)

[[5. 5. 0. ... 0. 0. 4.]
 [3. 0. 0. ... 0. 3. 0.]
 [3. 0. 0. ... 0. 4. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


Transferir el arreglo a GPU

In [8]:
inicio_tiempo = time.time()
matriz_gpu = gpuarray.to_gpu(matriz_df)
fin_load = time.time()

In [9]:
print('Tiempo para cargar datos en la GPU:', fin_load - inicio_tiempo)

Tiempo para cargar datos en la GPU: 0.04531288146972656


Ejecutamos SVD, originando 3 salidas (las matrices de descomposicion SVD) con la matriz que cargamos en GPU y especificamos que cuSolver sea el backend de nuestra operacion

In [10]:
gpu_svd_inicio = time.time()
U_gpu, s_gpu, V_gpu = culinalg.svd(matriz_gpu , lib='cusolver')
gpu_svd_fin = time.time()

In [11]:
print('Tiempo para realizar la SVD de una matriz en la GPU:',
      gpu_svd_fin-gpu_svd_inicio, 'sec')

Tiempo para realizar la SVD de una matriz en la GPU: 95.76512169837952 sec


Copiamos los arreglos resultantes de la GPU al host

In [12]:
load_datos_inicio = time.time()
U = U_gpu.get()
s = s_gpu.get()
V = V_gpu.get()
load_datos_fin = time.time()

In [13]:
print('Tiempo para realizar cargar resultados del device al host:',
      load_datos_fin-load_datos_inicio, 'sec')

Tiempo para realizar cargar resultados del device al host: 0.1815345287322998 sec


In [14]:
print('Total (incluyendo load host-device, device-host):', load_datos_fin - inicio_tiempo, 'sec')

Total (incluyendo load host-device, device-host): 96.06566619873047 sec


Convertir la matriz diagonal

In [15]:
S = np.zeros((3706,6040))
S[:3706,:3706] = np.diag(s)

Obtener matriz calculada

In [16]:
matriz_rec = np.dot(U, np.dot(S,V))

Revisar error comparado con matriz original

In [17]:
# Raíz del Error Cuadrático Medio (RMSE)
def RSME(pred,original):
    N = pred.shape[0] # filas
    M = pred.shape[1] # columnas
    cur_sum = np.sum(np.square(pred-original))
    return np.sqrt(cur_sum/(N*M))

In [18]:
# Coeficiente de correlación de Spearman (rho)
def rho(pred, original):
    num = np.sum(np.square(pred - original))
    n = pred.shape[0]*pred.shape[1]
    den = n*(n*n-1)
    return 1 - (6*num)/den

In [19]:
print("Error Cuadrático Medio", RSME(matriz_rec, matriz_df))
print("Coeficiente de correlación de Spearman", rho(matriz_rec, matriz_df))

Error Cuadrático Medio 1.0988685819247828
Coeficiente de correlación de Spearman 0.9999999999999856
